In [ ]:
!nvidia-smi



Sat Dec 20 05:41:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

检查是否连接到T4显卡

“！”后的为shell命令
卸载已经安装的kaggle包
kaggle包是做什么的--->命令行工具和Python接口：通过 Kaggle API 管理数据集、竞赛、提交、kernels（notebooks）
解释：命令行工具(cli)--->通过终端/命令行接受文本，执行特定任务的软件，便于交互--->用“文字命令”与操作系统或程序交互的界面
终端（Terminal）--->基于文本的界面，用来输入命令、运行程序并查看输出。底层通常运行一个 shell（常见有 bash、zsh）（用来解析命令）
Shell--->命令解释器（程序），把你在终端里输入的文本命令解析并执行---和操作系统交互


一些关于Shell的说明
Shell 是一个运行在用户态的程序（应用），职责是读取用户输入的文本命令、做词法/语法解析、执行命令（内建或外部）、管理作业（前台/后台）并返回结果。
普通程序（比如 ls、python、你的可执行程序）通常是被 Shell 启动的“外部命令”，执行具体任务并退出
解释/编排：Shell 本身是解释器/控制器（把文本命令变成系统调用和进程组合），而普通程序通常直接实现功能。
    状态与环境：Shell 保存会话状态（当前工作目录、环境变量、别名、函数、作业表），普通程序一般是一次性运行，状态更局部。
    组合能力：Shell 提供管道、重定向、命令替换、控制流（if/for/while），用于把小工具组合成复杂工作流。
    内建 vs 外部：一些命令（cd、export）必须是内建，因为它们要修改当前 Shell 的进程状态；其他程序可以是独立执行的外部可执行文件。
    系统调用：Shell 用 fork()/execve()/waitpid()/pipe()/dup2()/open()/close()/kill() 等系统调用来创建/管理子进程、设置 I/O、处理管道与信号。
    进程/作业控制：Shell 管理进程组、前台/后台任务，响应信号（SIGINT/SIGTSTP），并复用退出码做脚本判断。
    文件描述符：管道和重定向由内核的文件描述符机制实现（dup2 把子进程的 stdin/stdout 指向管道/文件）。
    权限/安全：执行二进制文件受内核权限与文件系统控制（UID/GID、可执行位、SELinux/AppArmor）
    用户态 vs 内核态：Shell 在用户态运行；每次 fork/exec 都涉及内核操作，CPU 在两者间切换（上下文切换）。
    程序执行流程：execve 将 ELF（可执行文件）加载到进程地址空间（代码段、数据段、堆、栈），CPU 按指令流（fetch-decode-execute）运行。
    中断/异常：键盘/终端输入、定时器、信号会引发中断或信号处理，Shell 需处理（例如 Ctrl+C 发 SIGINT）。
    性能影响：频繁创建进程/上下文切换有开销；因此 UNIX 设计了小工具+管道的组合思想以提高效率与可重用性。
sh、bash、zsh、fish、PowerShell（Windows）--->计目标、语法、交互体验、平台适配和脚本兼容性各不相同的Shell

“Python 接口（API）” 是什么

API = 应用程序编程接口（Application Programming Interface）。本质上是“约定/合同”：一组可调用的函数/方法、参数和返回值、以及错误/认证的约定。遵守这些约定就能用程序安全且可重复地使用服务。
对于 Kaggle，底层还有一个 HTTP（REST）接口（服务器端的端点），CLI 和 Python 库是该 REST 接口的客户端实现，负责认证（kaggle.json）、发请求、下载/提交并处理返回结果。

接口是“规范 + 实现”。规范定义了怎么调用（参数/返回/认证），实现（CLI/库）把规范变成你能直接用的工具并隐藏细节。

In [16]:
!pip -q uninstall -y kaggle
!pip -q install --no-cache-dir kaggle==1.5.16
!pip -q install --no-cache-dir pandas numpy torch tqdm
!kaggle --version


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


1.下载kaggle包
2.准备元数据（setup.py）...已完成
3.构建wheeel（可安装二进制包）
4.出现错误 ---> 调用了 authenticate() 去做认证检查 ---> 最终异常（OSError/IOError）：系统找不到 kaggle.json，程序因此终止。提示两种认证方法：把文件放到 /root/.kaggle（或 ~/.kaggle）或通过环境变量进行认证。

kaggle.json--->一个凭据文件（API token）：Kaggle 账号标识（username）和私密密钥（key）。Kaggle 的 CLI 和 Python 客户端读取它，用来通过 HTTPS 与 Kaggle 平台的 API 进行身份认证，从而允许下载数据、提交结果等操作。

json格式：量级的、基于文本的数据交换格式。它常用来在程序或网络服务之间传输结构化数据（例如 API 返回的数据、配置文件、凭据文件等）

注意的点：凭据类 JSON（例如 kaggle.json）包含秘密信息，不要把它提交到公共仓库或泄露。应设置恰当权限（如 chmod 600），并在不需要时移除或使用环境变量替代。

所以我们要上传kaggle.json文件,并放到指定的位置

在主目录下创建.kaggle目录
把当前目录的kaggle.json文件拷贝到.kaggle目录下
把 kaggle.json 的权限设置为 600--->这句没听懂，是什么意思？---> 

要补充知识了：
1.权限位--->Unix 权限用三位八进制表示：u g o（user/group/others）
每位是读（4）+写（2）+执行（1）的和
'''所以 600 等同于 rw-------（符号形式：u=rw, g=, o=）''',诶，为什么每一个数字都只能拆成几个确定的步骤之和，不会有歧义呢？


列出 ~/.kaggle 目录的详细内容（包含权限、文件大小、修改时间等），用于验证文件已复制且权限正确.

In [17]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -la ~/.kaggle


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
total 12
drwxr-xr-x 2 root root 4096 Dec 20 05:34 .
drwx------ 1 root root 4096 Dec 20 05:34 ..


1.当时没传kaggle.json
2.文件没有,所以改不了权限
3.目录中文件占用的“总块数”（文件系统统计），不是单个文件大小的含义
4..kaggle 目录确实已被创建，但目前没有用户文件（只有目录本身）
5.这是正常的用户主目录权限设置


所以肯定找不到 kaggle.json （因为就没有这个文件）

使用 Kaggle 的命令行工具 列出可用的竞赛，-s ml2022spring-hw1
-s 表示 search（按关键字筛选），这里只列出包含 ml2022spring-hw1 的竞赛条目。
把上一步的输出通过管道交给 head，只显示前 10 行（默认）

In [18]:
!kaggle competitions list -s ml2022spring-hw1 | head


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


当时没上传kaggle.json文件

In [ ]:
!pwd
!ls -la
!ls -la /content | head
!ls -la /content/kaggle.json || echo "没有在 /content 看到 kaggle.json"


/content/ml2022-spring
total 40
drwxr-xr-x 7 root root 4096 Dec 20 05:34 .
drwxr-xr-x 1 root root 4096 Dec 20 05:36 ..
drwxr-xr-x 3 root root 4096 Dec 20 05:34 assignments
drwxr-xr-x 4 root root 4096 Dec 20 05:34 docs
-rw-r--r-- 1 root root  180 Dec 20 05:34 .editorconfig
drwxr-xr-x 8 root root 4096 Dec 20 05:34 .git
-rw-r--r-- 1 root root  275 Dec 20 05:34 .gitignore
-rw-r--r-- 1 root root 1307 Dec 20 05:34 README.md
drwxr-xr-x 2 root root 4096 Dec 20 05:34 requirements
drwxr-xr-x 2 root root 4096 Dec 20 05:34 tools
total 32
drwxr-xr-x 1 root root 4096 Dec 20 05:36 .
drwxr-xr-x 1 root root 4096 Dec 20 05:28 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 2 root root 4096 Dec 20 05:35 data_hw1
drwx------ 5 root root 4096 Dec 20 05:34 drive
-rw-r--r-- 1 root root   63 Dec 20 05:36 kaggle.json
drwxr-xr-x 7 root root 4096 Dec 20 05:34 ml2022-spring
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data
-rw-r--r-- 1 root root 63 Dec 20 05:36 /content/kaggle.json


kaggle.json 文件确实存在在 /content，但之前的复制失败是因为当前工作目录是 /content/ml2022-spring，相对路径 kaggle.json 找不到文件。
接下来我给出逐行解释和修复命令

In [20]:
!mkdir -p /root/.kaggle

# 如果 kaggle.json 在当前目录：
!cp -f kaggle.json /root/.kaggle/kaggle.json 2>/dev/null || true

# 如果 kaggle.json 在 /content：
!cp -f /content/kaggle.json /root/.kaggle/kaggle.json 2>/dev/null || true

!chmod 600 /root/.kaggle/kaggle.json
!ls -la /root/.kaggle


total 16
drwxr-xr-x 2 root root 4096 Dec 20 05:45 .
drwx------ 1 root root 4096 Dec 20 05:34 ..
-rw------- 1 root root   63 Dec 20 05:45 kaggle.json


kaggle.json 已经出现在 ~/.kaggle，且权限为 600。

In [21]:
!kaggle --version
!kaggle competitions list -s ml2022spring-hw1 | head


Kaggle API 1.5.16
No competitions found


本地安装的 kaggle 客户端版本（client = 1.5.16）
提示客户端版本较旧（服务端是 1.8.0）。
搜索字符串没有找到匹配的竞赛

把你的 Google Drive 挂载到 Colab 虚拟机的文件系统（挂载点是 /content/drive）

In [22]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1.创建目录（含父目录），在挂载的 Google Drive 下准备保存数据集。
2.用 Kaggle CLI 从竞赛 ml2022spring-hw1 下载数据到指定目录（-c 指定竞赛，-p 指定下载路径）。
运行前需已配置好 kaggle.json 并接受竞赛规则/有权限。
3.解压目录下的 zip 文件到目标目录
4.列出目录内容（长格式、显示隐藏文件、人类可读大小），用于确认下载和解压结果。

In [23]:
!mkdir -p /content/drive/MyDrive/ml2022spring_hw1
!kaggle competitions download -c ml2022spring-hw1 -p /content/drive/MyDrive/ml2022spring_hw1
!unzip -o "/content/drive/MyDrive/ml2022spring_hw1/*.zip" -d /content/drive/MyDrive/ml2022spring_hw1
!ls -lah /content/drive/MyDrive/ml2022spring_hw1


  0% 0.00/1.00M [00:00<?, ?B/s]
100% 1.00M/1.00M [00:00<00:00, 67.2MB/s]
Archive:  /content/drive/MyDrive/ml2022spring_hw1/ml2022spring-hw1.zip
  inflating: /content/drive/MyDrive/ml2022spring_hw1/covid.test.csv  
  inflating: /content/drive/MyDrive/ml2022spring_hw1/covid.train.csv  
total 4.4M
-rw------- 1 root root 971K Feb 15  2022 covid.test.csv
-rw------- 1 root root 2.4M Feb 15  2022 covid.train.csv
-rw------- 1 root root 1.0M Feb 15  2022 ml2022spring-hw1.zip


-rw-------：权限位，表示属主有读写，组和其他无权限（即 600）。
1：硬链接数
root root：属主和属组都是 root
971K：显示的文件大小
Feb 15 2022：最后修改时间
最后是文件名

inode：包含权限、属主、时间戳、数据块指针等元数据，但不包含文件名。
目录项：目录里的条目，把名字绑定到某个 inode。多个目录项可以绑定同一 inode（这就是硬链接）

In [ ]:

'''
这段代码把 CSV 读入的 train_df / test_df 做格式对齐与检查：
确定 id 列和 target 列、把 test 的非 id 列作为特征列、
把训练/测试数据切成 numpy 数组并转为 float32，
最后打印维度信息，方便后续送入模型训练/预测。
'''


import os
import numpy as np
import pandas as pd

DATA_DIR = "/content/drive/MyDrive/ml2022spring_hw1"  
train_df = pd.read_csv(os.path.join(DATA_DIR, "covid.train.csv"))
test_df  = pd.read_csv(os.path.join(DATA_DIR, "covid.test.csv"))

print("train shape:", train_df.shape)#显示 train/test 的行列数（快速检查数据规模）。
print("test  shape:", test_df.shape)

# 推断 target：train 有但 test 没有的列--->找到要预测的列
diff = list(set(train_df.columns) - set(test_df.columns))
print("train-test diff columns:", diff)

target_col = diff[0] if len(diff) == 1 else train_df.columns[-1]
id_col = "id" if "id" in test_df.columns else test_df.columns[0]

# 以 test 的列（去掉 id）为 feature_cols，保证 train/test 完全对齐
feature_cols = [c for c in test_df.columns if c != id_col]

print("id_col:", id_col)
print("target_col:", target_col)
print("num_features:", len(feature_cols))

#以上为 确定 目标，对齐ID

# 构造数组
X = train_df[feature_cols].values.astype(np.float32)
# 从 train 中按 feature_cols 顺序取出特征矩阵并转为 float32（通常为节省内存并兼容 PyTorch）
y = train_df[target_col].values.astype(np.float32)
#取出训练目标向量并转为 float32
X_test = test_df[feature_cols].values.astype(np.float32)
#从 test 中按相同 feature_cols 取特征矩阵（确保排列一致），并转 float32。
test_ids = test_df[id_col].values

print("X:", X.shape, "X_test:", X_test.shape, "y:", y.shape)



'''我们是通过训练集，通过特征去预测目标，
然后以预测出的目标和真实的目标的关系为目标，
不断的"纠正“参数，然后在验证集使用这些参数进行预测，
并计算和验证集的评估效果。
最后提交模型参数和各种结构，
在kaggle平台在测试集实验，得到测试分数
'''

train shape: (2699, 118)
test  shape: (1078, 117)
train-test diff columns: ['tested_positive.4']
id_col: id
target_col: tested_positive.4
num_features: 116
X: (2699, 116) X_test: (1078, 116) y: (2699,)


训练集有 2699 行、118 列
测试集有 1078 行、117 列
列出在训练集但不在测试集中的列名，这里是 'tested_positive.4'---->目标列
X 是训练特征矩阵（2699 个样本 × 116 个特征）。
X_test 是测试特征矩阵（1078 × 116），与训练特征对齐。
y 是训练目标向量，长度 2699（每个训练样本对应一个目标值）

In [ ]:
'''这段代码用训练集统计量对特征做标准化（zero-mean, unit-variance），
把训练集按 80/20 随机划分为训练/验证两部分，
构造 PyTorch 的 TensorDataset 与 DataLoader（以 batch 方式供模型训练）。
标准化只用训练集的 mean/std，避免信息泄露；unsqueeze(1) 把 y 变成 (N,1) 以匹配回归模型输出。

'''
import torch
from torch.utils.data import DataLoader, TensorDataset

# 标准化（只用训练集统计量）
mean = X.mean(axis=0, keepdims=True)#计算均值
std  = X.std(axis=0, keepdims=True) + 1e-6#计算标准差，并防止除0

Xn = (X - mean) / std
Xn_test = (X_test - mean) / std#注意不要用测试集的统计量

# train/valid split
N = len(Xn)
idx = np.arange(N)#索引数组
np.random.seed(42)#随机种子
np.random.shuffle(idx)#打乱索引
split = int(N * 0.8)
tr_idx, va_idx = idx[:split], idx[split:]#划分验证集与训练集

X_tr, y_tr = Xn[tr_idx], y[tr_idx]#训练集的特征和真值
X_va, y_va = Xn[va_idx], y[va_idx]#验证集的特征和真值


#分批次学习
train_loader = DataLoader(
    TensorDataset(torch.from_numpy(X_tr), torch.from_numpy(y_tr).unsqueeze(1)),
    batch_size=128, shuffle=True
)

'''
DataLoader 就像是一个“分发器”，它会把整个数据集分成小批次，每次取出一小块数据。

batch_size=128：每次取 128 个样本。
shuffle=True：在每次训练开始时，随机打乱数据的顺序，
避免模型总是以相同的顺序看到数据。
'''
#unsqueeze(1)：调整目标值的形状,unsqueeze(1) 会把它变成二维的，形状是 (N, 1)
#为什么要这样做？因为 PyTorch 的模型输出通常是二维的，比如 (batch_size, 1)，所以目标值的形状也需要匹配。
'''
把 numpy 数组转为 torch.Tensor（dtype 为 float32），
用 TensorDataset 将特征与标签配对，
再用 DataLoader 以 batch_size=128、每 epoch 打乱（shuffle=True）返回训练批次。
y_tr.unsqueeze(1) 把 y 形状从 (B,) 变为 (B,1)，与模型输出 (B,1) 对齐。
'''
valid_x = torch.from_numpy(X_va)
valid_y = torch.from_numpy(y_va).unsqueeze(1)
#对验证集的处理

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)


device: cuda


In [26]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, in_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        return self.net(x)

model = MLP(Xn.shape[1]).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
loss_fn = nn.MSELoss()

valid_x = valid_x.to(device)
valid_y = valid_y.to(device)

best_mse = float("inf")
best_state = None
patience = 10
wait = 0

for epoch in range(1, 201):  # 200 epochs 上限
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = loss_fn(pred, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()

    # validate
    model.eval()
    with torch.no_grad():
        va_pred = model(valid_x)
        va_mse = loss_fn(va_pred, valid_y).item()

    if epoch % 10 == 0:
        print(f"epoch {epoch:03d} | valid MSE: {va_mse:.6f}")

    # early stopping
    if va_mse < best_mse - 1e-6:
        best_mse = va_mse
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stop at epoch {epoch}, best valid MSE = {best_mse:.6f}")
            break

# load best
model.load_state_dict(best_state)
model = model.to(device)
print("best valid MSE:", best_mse)


epoch 010 | valid MSE: 2.323007
epoch 020 | valid MSE: 1.700079
epoch 030 | valid MSE: 1.451448
epoch 040 | valid MSE: 1.340536
epoch 050 | valid MSE: 1.328585
epoch 060 | valid MSE: 1.479543
epoch 070 | valid MSE: 1.190755
Early stop at epoch 75, best valid MSE = 1.187714
best valid MSE: 1.1877137422561646


In [27]:
model.eval()
with torch.no_grad():
    preds = model(torch.from_numpy(Xn_test).to(device)).cpu().numpy().reshape(-1)

sub = pd.DataFrame({id_col: test_ids, target_col: preds})
out_path = "/content/submission.csv"
sub.to_csv(out_path, index=False)
print("saved:", out_path)
sub.head()


saved: /content/submission.csv


,id,tested_positive.4
0,0,7.965214
1,1,8.578969
2,2,4.691366
3,3,8.281577
4,4,16.618568


In [28]:
!kaggle competitions submit -c ml2022spring-hw1 -f /content/submission.csv -m "HW01 MLP baseline"


100% 14.4k/14.4k [00:00<00:00, 21.9kB/s]
Successfully submitted to ML2022Spring-hw1

In [29]:
!pip freeze > /content/requirements_lock_hw01.txt


In [30]:
!kaggle competitions submissions -c ml2022spring-hw1 | head -n 10


fileName        date                 description                          status  publicScore  privateScore  
--------------  -------------------  -----------------------------------  ------  -----------  ------------  
submission.csv  2025-12-20 05:52:57  HW01 MLP baseline                    error                              
submission.csv  2025-12-19 15:04:53  HW01 baseline: linear + standardize  error                              


In [31]:
import pandas as pd
import numpy as np

sub_path = "/content/submission.csv"
sub = pd.read_csv(sub_path)

print("shape:", sub.shape)
print("columns:", list(sub.columns))
print("head:\n", sub.head())
print("NaN count:\n", sub.isna().sum())
print("finite preds:", np.isfinite(sub.iloc[:,1].to_numpy()).all())


shape: (1078, 2)
columns: ['id', 'tested_positive.4']
head:
    id  tested_positive.4
0   0           7.965214
1   1           8.578969
2   2           4.691366
3   3           8.281577
4   4          16.618568
NaN count:
 id                   0
tested_positive.4    0
dtype: int64
finite preds: True


In [32]:
import pandas as pd
import numpy as np

SUBMIT_COL = "tested_positive"   # 关键：提交列名用 tested_positive
ID_COL = "id"

# 确保 id 是 int 且与 test 对齐
ids = test_df[ID_COL].to_numpy()
preds_np = np.asarray(preds, dtype=np.float32)

sub2 = pd.DataFrame({ID_COL: ids, SUBMIT_COL: preds_np})

# 常见要求：按 id 排序（稳妥）
sub2 = sub2.sort_values(ID_COL).reset_index(drop=True)

# 再做一次健壮性检查
assert sub2.shape[0] == test_df.shape[0]
assert sub2[ID_COL].isna().sum() == 0
assert np.isfinite(sub2[SUBMIT_COL].to_numpy()).all()

out_path = "/content/submission.csv"
sub2.to_csv(out_path, index=False)
print("saved:", out_path)
print(sub2.head())
print("columns:", list(sub2.columns))


saved: /content/submission.csv
   id  tested_positive
0   0         7.965214
1   1         8.578969
2   2         4.691366
3   3         8.281577
4   4        16.618568
columns: ['id', 'tested_positive']


In [33]:
!kaggle competitions submit -c ml2022spring-hw1 -f /content/submission.csv -m "HW01 MLP baseline (fixed submission header)"


100% 14.4k/14.4k [00:00<00:00, 22.4kB/s]
Successfully submitted to ML2022Spring-hw1

In [34]:
!kaggle competitions submissions -c ml2022spring-hw1 | head -n 10


fileName        date                 description                                  status    publicScore  privateScore  
--------------  -------------------  -------------------------------------------  --------  -----------  ------------  
submission.csv  2025-12-20 05:57:47  HW01 MLP baseline (fixed submission header)  complete  9.09168      7.84961       
submission.csv  2025-12-20 05:52:57  HW01 MLP baseline                            error                                
submission.csv  2025-12-19 15:04:53  HW01 baseline: linear + standardize          error                                
